<a href="https://colab.research.google.com/github/agpo-ilr-uni-bonn/PromotionskollegModule6800_2024/blob/master/6800_Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 2: Code used during lecture and lab assignment

## Instructions

- The notebook combines 'code used during lecture' with the 'Day 2 lab' assignment (see further down)
- The lab assignment can be done largely by copying/paste/modification of the code used during the lecture
- Please add answers/discussion/comments to the notebook as comments or text box. Do not create another file in addition.
- When you are done with your assignment, save the notebook in drive and add your last name to the name of the file.
- To hand in the final notebook follow the instructions provided by email



# **Code used during lecture**
## Part One: Tree based methods

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn import tree
import os

In [ ]:
# Set the numpy random seed
np.random.seed(100)

In [ ]:
# run this cell only once if you don't have wget installed
# its assumed you are using windows and have python installed
# only needed if you are running the notebook locally
# %pip install wget
#if not os.path.isfile('brazil_all_data_v2.gz'):
#    !python -m wget  https://ilr-ml.s3.eu-central-1.amazonaws.com/brazil_all_data_v2.gz
# Download data only once and make sure it is in the same folder as the notebook

# check if brazil_all_data_v2.gz is available in the current folder and if not, download it

if not os.path.isfile('brazil_all_data_v2.gz'):
    !wget  https://ilr-ml.s3.eu-central-1.amazonaws.com/brazil_all_data_v2.gz


In [ ]:
# Load data with pandas into a dataframe
df = pd.read_parquet('brazil_all_data_v2.gz')

In [ ]:
# Define binary variable for deforestration in 2018
df['D_defor_2018'] = df['defor_2018']>0
Y_all = df['D_defor_2018']

In [ ]:
# Define a list of features names (explantory variables)
lstX = [
  'wdpa_2017',
  'population_2015',
  'chirps_2017',
  'defor_2017',
  'maize',
  'soy',
  'sugarcane',
  'perc_treecover',
  'perm_water',
  'travel_min',
  'cropland',
  'mean_elev',
  'sd_elev',
  'near_road',
  'defor_2017_lag_1st_order',
  'wdpa_2017_lag_1st_order',
  'chirps_2017_lag_1st_order',
  'population_2015_lag_1st_order',
  'maize_lag_1st_order',
  'soy_lag_1st_order',
  'sugarcane_lag_1st_order',
  'perc_treecover_lag_1st_order',
  'perm_water_lag_1st_order',
  'travel_min_lag_1st_order',
  'cropland_lag_1st_order',
  'mean_elev_lag_1st_order',
  'sd_elev_lag_1st_order',
  'near_road_lag_1st_order',
 ]

# Get the explanatory Variables
X_all =  df.loc[:,lstX]

In [ ]:
# Split the data into train and test data using sklearn train_test_split object
#   (see: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

#   Note: This randomly split the data in 80% train and 20% test data
X_train_raw, X_test_raw, Y_train, Y_test = train_test_split(X_all, Y_all, test_size = 0.2)

In [ ]:
# Scale data to 0-1 range using sklearn MinMaxScalar object
# (see: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)
scaler = MinMaxScaler()
# Use only the train data to fit the MinMaxScalar
scaler.fit(X_train_raw)

# Apply the MinMax transformation to the train and test data
X_train = scaler.transform(X_train_raw)
X_test = scaler.transform(X_test_raw)
# Note the depended variable does not need to be scaled as it is a binary variable anyway

Run logit on deforestation binary variable

In [ ]:
# Fit a logistic regression model using sklearn
# (see: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

# Create the model object
modelLg = LogisticRegression(random_state=0,penalty=None,fit_intercept=True,max_iter=1000)
# Fit the model using the training data
modelLg.fit(X_train, Y_train)

In [ ]:
# Define a function that prints the model statistics.
# We will use the function below to always get the same model stats for each of
# the models the we will estimate below.
def printOutput(mod,X_train,Y_train,X_test,Y_test):
  # view results
  print('Score in train', mod.score(X_train, Y_train))
  print('Score in test', mod.score(X_test, Y_test))

  Y_test_had_Tree = mod.predict(X_test)

  print('\nConfusion Matrix')
  print(pd.DataFrame(confusion_matrix(Y_test, Y_test_had_Tree),
            index=pd.MultiIndex.from_arrays([['actual','actual'], ['False','True']]),
            columns=pd.MultiIndex.from_arrays([['predicted','predicted'], ['False','True']])))

In [ ]:
# Use the function to print the model statistics for our logit model
printOutput(modelLg,X_train,Y_train,X_test,Y_test)

# Compute ROC curve and ROC area for each class

# Get the predicted probabiltities
Y_score = modelLg.decision_function(X_test)

# Get true positive and false positive rate
# See: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html
fpr_Lg, tpr_Lg, _ = roc_curve(Y_test, Y_score)

# Get the Area under the cureve (AUC)
# See: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html
roc_auc_Lg = auc(fpr_Lg, tpr_Lg)

print('\nROC AUC', roc_auc_Lg)

In [ ]:
# Plot the ROC curve
plt.figure()
lw = 2
plt.plot(fpr_Lg, tpr_Lg, color='darkorange',
         lw=lw, label='Logistic ROC curve (area = %0.2f' % roc_auc_Lg)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

Now run a decision tree using the same specification

In [ ]:
# Fit a decision tree using sklearn
# (see https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

# Define a model object
modelTree = tree.DecisionTreeClassifier()
# Fit the model
modelTree = modelTree.fit(X_train, Y_train)

In [ ]:
# Use the function to print the model statistics for our tree model
printOutput(modelTree,X_train,Y_train,X_test,Y_test)

Run the same model using  a random forest

In [15]:
# run a random forest using sklearn and default hyperparameters (note, this will take a few minutes)
# (see https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
from sklearn.ensemble import RandomForestClassifier

# Create model object
# Note: We reduce the number of estimators here to speed-up runtime,
#       default value for n_estimators is 100
modelForest = RandomForestClassifier(n_estimators=20)
# Fit model
modelForest = modelForest.fit(X_train, Y_train)

In [16]:
# Print model output stats
printOutput(modelForest,X_train,Y_train,X_test,Y_test)


Score in train 0.995879010962631
Score in test 0.8298991758021925

Confusion Matrix
             predicted      
                 False  True
actual False     35565  2374
       True       6129  5920


In [ ]:
# ====================
# Discuss in the group
# ====================
# What do you conclude from the model outcome. Is this a
# useful model. Compare the results to the logit outcomes.

In [ ]:
# Plot ROC curve
# Get the predicted probabiltities
Y_scoreRF = modelForest.predict_proba(X_test)[:,1]

# Get true positive and false positive rate
fpr_RF, tpr_RF, _ = roc_curve(Y_test, Y_scoreRF)

# Get the Area under the cureve (AUC)
roc_auc_RF = auc(fpr_RF, tpr_RF)


plt.figure()
lw = 2
plt.plot(fpr_RF, tpr_RF,
         lw=lw, label='RF ROC curve (area = %0.2f' % roc_auc_RF)
plt.plot(fpr_Lg, tpr_Lg,
         lw=lw, label='Logistic ROC curve (area = %0.2f' % roc_auc_Lg)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

Visualizing the results

In [ ]:
# Generate a feature importance graph for the forest
# Adjusted based on  https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html

importances = modelForest.feature_importances_
std = np.std([tree.feature_importances_ for tree in modelForest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")


for f in range(X_train.shape[1]):
    print("%d. %s (%f)" % (f + 1, lstX[f], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
# plt.xticks(range(X_train.shape[1]), indices)
plt.xlim([-1, X_train.shape[1]])
plt.show()

## Part Two: SHAP values

In [ ]:
# First install the SHAP libary
!pip install -q shap

In [ ]:
# Import the shape libary
import shap
# Load JS visualization code to notebook
shap.initjs()

In [ ]:
# Create a dataframe for our train data that includes the variable names
df_X_train = pd.DataFrame(X_train,columns=lstX)

# Explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(modelForest)

In [ ]:
df_X_train_subsample = shap.sample(df_X_train, 100)

In [ ]:
# Calculate the shape values using the TreeExplainer object
shap_values = explainer.shap_values(df_X_train_subsample)

In [ ]:
# Get predicted probabilities to compare shape values
# Y_train_proba = modelTree.predict_proba(df_X_train_subsample)
Y_train_proba = modelForest.predict_proba(df_X_train_subsample)

In [ ]:
# Inspect shape of SHAP values
shap_values.shape


In [ ]:
plt.hist(shap_values[lstX.index('defor_2017'),:,1]);

In [ ]:
# visualize the first prediction's explanation
# shap.force_plot(explainer.expected_value, shap_values[0,:], df_X_train.iloc[0,:])
iobs = 0
# If you have a javascript error use matplotlib=True to avoid Javascript
shap.force_plot(explainer.expected_value[0], shap_values[iobs,:,1],
                np.round(df_X_train_subsample.iloc[iobs,:],4),matplotlib=True,
                contribution_threshold=0.1)
# Note: This might look different than the version the slides because another random seed
#       was used to create the plots in the slides

In [ ]:
# Explore shape values manually
shap_iobs = explainer.expected_value + shap_values[iobs,:].sum()
print(f'SHAP value of obs {iobs}:',shap_iobs)
print('SHAP value for base:', explainer.expected_value)

# SHAP value are in log-odds, transform to probability
print('Expected Prob:',np.exp(explainer.expected_value) / (1 + np.exp(explainer.expected_value)))
print(f'Prob of obs {iobs}:',np.exp(shap_iobs) / (1 + np.exp(shap_iobs)))

# compare to predicted proability
print(f'Proba of obs {iobs}:', Y_train_proba[iobs][1])

# **Lab**
## Part One: Tree based methods

The lab today will have you predict deforestation using both a random forest and XGboost models

In [ ]:
# In the lecture part we have run a Random Forest that heavily
# overfitted the training data. Adjust a hyperparamter
# and see if you can train a model that does not overfit.

# Hint: Vary the parameter max_depth or min_samples_split

# Sklearn documentation on RF:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict_proba

# ==============
# Your code here
# ==============
modelForest = ...

# Fit model
...

#  Print model stats output
printOutput(...)

In [ ]:
# Now run an XGBoost model for the same task
import xgboost as xgb

# Hint: 1) Look at one of the tutorial on XGB on how to specify and run the model
#          (e.g. https://hackernoon.com/want-a-complete-guide-for-xgboost-model-in-python-using-scikit-learn-sc11f31bq)
#       2) You do not need to adjust hypterparamter here. The default paramters
#          should be fine here.

# ==============
# Your code here
# ==============
# Specify model
model_xgb = ...


# Fit model to data
model_xgb ...


In [ ]:
# Print the model stats of you XGB model using the function from above
# ==============
# Your code here
# ==============
...



In [ ]:
# Compare to the outcome of the other model
# (not need to change anything here)
print('\n--- Logistic')
printOutput(modelLg,X_train,Y_train,X_test,Y_test)

print('\n--- Tree')
printOutput(modelTree,X_train,Y_train,X_test,Y_test)

print('\n--- Forest')
printOutput(modelForest,X_train,Y_train,X_test,Y_test)


In [ ]:
# Write a couple of sentences on what you see in your models, which you
# believe are performing better and why
# ================
# Your answer here
# ================




In [ ]:
# Plot an ROC corve for the Logit, Random Forest and XGB model

# Hint: Check what we have done above...
# ==============
# Your code here
# ==============
# Get the predicted probabiltities
Y_scoreXG = ...

# Get true positive and false positive rate
fpr_XG, tpr_XG, _ = ...

# Get the Area under the cureve (AUC)
roc_auc_XG = ...



In [ ]:
# Plot the figure (not need to change anything here)
plt.figure()
lw = 2

plt.plot(fpr_XG, tpr_XG,
         lw=lw, label='XG ROC curve (area = %0.2f' % roc_auc_XG)

plt.plot(fpr_Lg, tpr_Lg,
         lw=lw, label='Logistic ROC curve (area = %0.2f' % roc_auc_Lg)

plt.plot(fpr_RF, tpr_RF,
         lw=lw, label='RF ROC curve (area = %0.2f' % roc_auc_RF)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# In one or two sentences, discuss what the ROC curves are telling you
# ================
# Your answer here
# ================



### Optional Tasks

(Optional) Generate your model's prediction errors and explore them - comparing different subsets of your data (e.g. protected areas vs others)

## Lab Part Two: SHAP Values
The second part focus on SHAP values. First we show you how to plot SHAP values for the XG Boost model, which you have already seen in the lecutre. Then you  should create SHAP values for the logit model and explore how SHAP value would look like in a well known linear model.

More on SHAP values:

https://github.com/slundberg/shap


**First run the code for SHAP values from the Lecture, then continue with this part**

In [ ]:
# ====================
# Discuss in the group
# ====================
# How do you interpret this plot?
# When might you want to use a plot like this?

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values[:,:,1], df_X_train_subsample)

In [ ]:

# ====================
# Discuss in the group
# ====================
# How do you interpret this plot?

In [ ]:
# To rrproduce the plot in the lecture use a larger subsample size
# Note, that computation takes longer the larger the subsample size
df_X_train_subsample = shap.sample(df_X_train, 1000)
shap_values = explainer.shap_values(df_X_train_subsample)

In [ ]:
# create a dependence plot to show the effect of a single feature across the whole dataset
shap.dependence_plot("defor_2017", shap_values[:,:,1], df_X_train_subsample)

In [ ]:
# ====================
# Discuss in the group
# ====================
# How do you interpret this plot?

In [ ]:
# ====================
# Discuss in the group
# ====================
# Before you start creating the plots, discuss in the group what kind of
# results you expect for the linear model. Specifically, think about how the
# last plot (the dependence_plot) would look in this case.

In [ ]:
# Here we use the shap.LinearExplainer() function instead of the
# shap.TreeExplainer(...) we used above

# You can have a look here for a reference:
# https://slundberg.github.io/shap/notebooks/linear_explainer/Sentiment%20Analysis%20with%20Logistic%20Regression.html

# ==============
# Your code here
# ==============
explainer = ...

shap_values = ...

In [ ]:
# Now create a dependence plot to show the effect of a single feature
# across the whole dataset, as was done above but now for the logit model

# ==============
# Your code here
# ==============
 ...

In [ ]:
# ====================
# Discuss in the group
# ====================


# 1) Does this look like to what you expected?

# 2) How does this compare to the plot for the XGB model. What can you conclude?

# Note: Below you find the usual regression output for logit model again. This
#       might be interesting as a reference.

In [ ]:

# Have a look at the SHAP summary plot
shap.summary_plot(shap_values, df_X_train)

## **For your reference**

Lets create our usual regression output for the logit model as a reference

This used the same code from the lab intro session.

(Not need to change/do anything here)

In [ ]:
from scipy.stats import norm
# Function to calculate pvalues and standard errors for a scikit-learn logisticRegression
# Source: https://stackoverflow.com/questions/25122999/scikit-learn-how-to-check-coefficients-significance
def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    # m = len(model.coef_[0])
    # coefs = model.coef_[0]
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se
    p = (1 - norm.cdf(abs(t))) * 2
    return se, p

In [ ]:
# Use the previously created function to create a regression output table
se, p = logit_pvalue(modelLg, X_train)
coefs = np.concatenate([modelLg.intercept_, modelLg.coef_[0]]).T
resCoef = pd.DataFrame(coefs,index=['constant']+lstX)
resCoef.columns = ['coef']
resCoef['se'] = se
resCoef['pval'] = p
resCoef

### **Optional Tasks**


In [ ]:
# (Optional-1) Explore the Shapley Value Explanations for different sub-sets of the data (e.g. protected areas versus others)
#  and in a few sentences, discuss your findings
#================

#================